# Celosia - Dynamic network update demo

In [1]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.python.client import device_lib
import time
import seaborn as sns
import matplotlib.gridspec as gridspec

from keras.layers import Input, Dense
from keras import regularizers, Model
from keras.models import Sequential, model_from_json
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import RMSprop

from sklearn.manifold import TSNE
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_recall_curve, classification_report, confusion_matrix, precision_score, average_precision_score, roc_curve, auc, recall_score, f1_score

Using TensorFlow backend.


In [2]:
# Import benign and gafgyt combo dataset for a given device id (1-9)
def import_dataset_benign_gagfyt_combo(device_id):
    normal = pd.read_csv('../input/nbaiot-dataset/{}.benign.csv'.format(device_id))
    n_X = normal.iloc[:,]
    n_X_scaled = MinMaxScaler().fit_transform(n_X.values)
    n_y = np.ones(n_X.shape[0]) # 1 represents normal

    anomalous = pd.read_csv('../input/nbaiot-dataset/{}.gafgyt.combo.csv'.format(device_id))
    a_X = anomalous.iloc[:,]
    a_X_scaled = MinMaxScaler().fit_transform(a_X.values)
    a_y = np.zeros(a_X.shape[0]) # 0 represents anomalous

    #normal.info()
    #normal.describe()
    #normal.head()

    #anomalous.info()
    #anomalous.describe()
    #anomalous.head()

    return (n_X_scaled, n_y, a_X_scaled, a_y)

In [3]:
def build_model(feature_count):
    model = Sequential()
    model.add(Dense(units=8, kernel_initializer="uniform", activation="relu", input_dim=feature_count)) # Hidden Layer 1 with 8 nodes
    model.add(Dense(units=6, kernel_initializer="uniform", activation="relu"))  # Hidden Layer 2 with 6 nodes
    model.add(Dense(units=1, kernel_initializer="uniform", activation="sigmoid")) # Output Layer
    model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [4]:
def change_model_feature_count(model, feature_count):
    # replace input shape of first layer
    model._layers[1].batch_input_shape = (None, feature_count)

    # rebuild model architecture by exporting and importing via json
    new_model = model_from_json(model.to_json())
    #new_model.summary()

    # copy weights from old model to new one
    for layer in new_model.layers:
        try:
            layer.set_weights(model.get_layer(name=layer.name).get_weights())
        except:
            print("Could not transfer weights for layer {}".format(layer.name))

    new_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return new_model

In [5]:
def train_model(model, X_train, y_train):
    history = model.fit(X_train, y_train,
                    batch_size=64,
                    epochs=3) #,
                    # We pass some validation for
                    # monitoring validation loss and metrics
                    # at the end of each epoch
                    #validation_data=(X_test, y_test))

In [6]:
def evaluate_model(model, X_test, y_test):
    #model.summary()

    y_pred = model.predict(X_test)
    y_pred = y_pred.round()

    #print ("")
    #print ("Classification Report: ")
    #print (classification_report(y_test, y_pred.round()))

    #print ("")
    #print ("Accuracy Score: ", accuracy_score(y_test, y_pred.round()))
    #loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    return (accuracy, precision, recall, f1)

In [7]:
def limit_dataset_feature_count(feature_count, n_X_scaled, n_y, a_X_scaled, a_y):
    X = np.append(n_X_scaled[:,:feature_count], a_X_scaled[:,:feature_count], axis = 0)
    y = np.append(n_y, a_y)

    return train_test_split(X, y, test_size=0.25)

In [8]:
def print_accuracy_metrices(title, metrices):
    (accuracy, precision, recall, f1) = metrices
    print (f"{title}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")

In [9]:
def dynamic_network_expansion(device_id):
    devices_names = ['Danmini_Doorbell', 'Ecobee_Thermostat', 'Ennio_Doorbell', 'Philips_B120N10_Baby_Monitor', 'Provision_PT_737E_Security_Camera', 'Provision_PT_838_Security_Camera', 'Samsung_SNH_1011_N_Webcam', 'SimpleHome_XCS7_1002_WHT_Security_Camera', 'SimpleHome_XCS7_1003_WHT_Security_Camera']
    (n_X_scaled, n_y, a_X_scaled, a_y) = import_dataset_benign_gagfyt_combo(device_id)
    feature_count = 100
    X_train, X_test, y_train, y_test = limit_dataset_feature_count(feature_count, n_X_scaled, n_y, a_X_scaled, a_y)

    model = build_model(feature_count)

    train_model(model, X_train, y_train)

    evaluate_model(model, X_test, y_test)

    new_feature_count = 115
    X_train, X_test, y_train, y_test = limit_dataset_feature_count(new_feature_count, n_X_scaled, n_y, a_X_scaled, a_y)
    new_model = change_model_feature_count(model, new_feature_count)

    before = evaluate_model(new_model, X_test, y_test)
    train_model(new_model, X_train, y_train)
    after = evaluate_model(new_model, X_test, y_test)

    return (devices_names[device_id - 1], before, after)

In [10]:
device_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9]
metrices = []
for device_id in device_ids:
    metrices.append(dynamic_network_expansion(device_id))

Epoch 1/3
81949/81949 [==============================] - 2s 30us/step - loss: 0.0786 - accuracy: 0.9661
Epoch 2/3
81949/81949 [==============================] - 2s 24us/step - loss: 0.0014 - accuracy: 0.9997
Epoch 3/3
81949/81949 [==============================] - 2s 26us/step - loss: 6.2130e-04 - accuracy: 0.9999
Could not transfer weights for layer dense_1
Epoch 1/3
81949/81949 [==============================] - 2s 27us/step - loss: 0.0267 - accuracy: 0.9962
Epoch 2/3
81949/81949 [==============================] - 2s 26us/step - loss: 6.9174e-04 - accuracy: 0.9999
Epoch 3/3
81949/81949 [==============================] - 2s 26us/step - loss: 3.6508e-04 - accuracy: 0.9999
Epoch 1/3
49593/49593 [==============================] - 1s 28us/step - loss: 0.1253 - accuracy: 0.9730
Epoch 2/3
49593/49593 [==============================] - 1s 25us/step - loss: 0.0017 - accuracy: 0.9998
Epoch 3/3
49593/49593 [==============================] - 1s 25us/step - loss: 7.4728e-04 - accuracy: 0.9998
Cou

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 1/3
49593/49593 [==============================] - 1s 28us/step - loss: 0.0492 - accuracy: 0.9840
Epoch 2/3
49593/49593 [==============================] - 1s 26us/step - loss: 8.8812e-04 - accuracy: 0.9999
Epoch 3/3
49593/49593 [==============================] - 1s 25us/step - loss: 3.7009e-04 - accuracy: 0.9999
Epoch 1/3
69085/69085 [==============================] - 2s 27us/step - loss: 0.0820 - accuracy: 0.9778
Epoch 2/3
69085/69085 [==============================] - 2s 25us/step - loss: 0.0017 - accuracy: 0.9996
Epoch 3/3
69085/69085 [==============================] - 2s 26us/step - loss: 6.3058e-04 - accuracy: 0.9998
Could not transfer weights for layer dense_7
Epoch 1/3
69085/69085 [==============================] - 2s 27us/step - loss: 0.0271 - accuracy: 0.9986
Epoch 2/3
69085/69085 [==============================] - 2s 25us/step - loss: 4.4285e-04 - accuracy: 1.0000
Epoch 3/3
69085/69085 [==============================] - 2s 26us/step - loss: 1.3940e-04 - accuracy: 1.0000

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 1/3
117033/117033 [==============================] - 3s 27us/step - loss: 0.0640 - accuracy: 0.9796
Epoch 2/3
117033/117033 [==============================] - 3s 25us/step - loss: 0.0085 - accuracy: 0.9999
Epoch 3/3
117033/117033 [==============================] - 3s 26us/step - loss: 0.0048 - accuracy: 1.0000
Epoch 1/3
83114/83114 [==============================] - 2s 28us/step - loss: 0.0659 - accuracy: 0.9881
Epoch 2/3
83114/83114 [==============================] - 2s 26us/step - loss: 9.6123e-04 - accuracy: 0.9999
Epoch 3/3
83114/83114 [==============================] - 2s 25us/step - loss: 4.8408e-04 - accuracy: 0.9999
Could not transfer weights for layer dense_19
Epoch 1/3
83114/83114 [==============================] - 2s 27us/step - loss: 0.0319 - accuracy: 0.9972
Epoch 2/3
83114/83114 [==============================] - 2s 26us/step - loss: 3.9180e-04 - accuracy: 1.0000
Epoch 3/3
83114/83114 [==============================] - 2s 26us/step - loss: 1.6103e-04 - accuracy: 1.0

In [11]:
for metric in metrices:
    (name, before, after) = metric
    print(name)
    print("")
    print_accuracy_metrices(f"{name}-Before", before)
    print_accuracy_metrices(f"{name}-After", after)
    print("-----------------")

Danmini_Doorbell

Danmini_Doorbell-Before, Accuracy: 0.4503056704616173, Precision: 0.45024529545288133, Recall: 1.0, F1: 0.620922952640614
Danmini_Doorbell-After, Accuracy: 1.0, Precision: 1.0, Recall: 1.0, F1: 1.0
-----------------
Ecobee_Thermostat

Ecobee_Thermostat-Before, Accuracy: 0.8024437454633438, Precision: 0.0, Recall: 0.0, F1: 0.0
Ecobee_Thermostat-After, Accuracy: 1.0, Precision: 1.0, Recall: 1.0, F1: 1.0
-----------------
Ennio_Doorbell

Ennio_Doorbell-Before, Accuracy: 0.40492422597594335, Precision: 0.4119426893298923, Recall: 0.94992817566181, F1: 0.5746741154562383
Ennio_Doorbell-After, Accuracy: 1.0, Precision: 1.0, Recall: 1.0, F1: 1.0
-----------------
Philips_B120N10_Baby_Monitor

Philips_B120N10_Baby_Monitor-Before, Accuracy: 0.7541989442654419, Precision: 0.7541989442654419, Recall: 1.0, F1: 0.8598784610274146
Philips_B120N10_Baby_Monitor-After, Accuracy: 0.9999828614519778, Precision: 0.9999772763424001, Recall: 1.0, F1: 0.9999886380421075
-----------------
Pr